In [1]:
import pandas as pd
import numpy as np
from BinanceCollector import BinanceCollector
from TradeBook import TradeBook
import warnings
warnings.filterwarnings("ignore")
from BreakdownCalculator import BreakdownCalculator
from BinanceSync import BinanceSync
from HyperBacktest import HyperBacktest
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from SlackCron import SlackCron

c:\Users\deves\AppData\Local\Programs\Python\Python37\lib\site-packages\requests\__init__.py:114: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning,


In [2]:
start = (datetime.now() - timedelta(days=60)).strftime("%Y-%m-%d")
end = (datetime.now()-timedelta(days=1)).strftime("%Y-%m-%d")
symbol = "ETHUSDT"
interval = "15m"

In [3]:
collector = BinanceCollector(symbol,start,end,interval)
collector.start_collect()
collector.df.columns = ["Open Time","Open","High","Low","Close","Volume","Close Time",
             "Quote asset volumne","Total Trades","Buyer Base Volume",
             "Taker buy quote asset volume","Ignore"]
df = collector.df
df["Open Time"] = pd.to_datetime(df["Open Time"],unit="ms")
df["Close Time"] = pd.to_datetime(df["Close Time"],unit="ms")
target = 4
sl = 2

In [8]:
cron = SlackCron()

In [6]:
done = False
while True:
    # Change according to interval
    if (datetime.now().minute-1)%15 != 0:
        done = False
        continue
    if done:
        continue
    print("Startng sync...")
    sync = BinanceSync(df,symbol,interval)
    sync.sync()
    df = sync.df
    df["EMA_20"] = df["Close"].ewm(span=20,adjust=False).mean()
    df["EMA_50"] = df["Close"].ewm(span=50,adjust=False).mean()
    df["EMA_200"] = df["Close"].ewm(span=200,adjust=False).mean()
    df["Buy"] = np.where( (df["EMA_20"] > df["EMA_50"]) & (df["EMA_20"].shift(1) <= df["EMA_50"].shift(1) ),1,0 )
    df["Sell"] = np.where( (df["EMA_20"] < df["EMA_50"]) & (df["EMA_20"].shift(1) >= df["EMA_50"].shift(1) ),1,0 )
    print(df.iloc[-1])
    print("Synced!")
    time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data = {
        'time':time
    }
    if df.iloc[-1]["Buy"] == 1:
        print(f"Buy Signal at {df.iloc[-1]['Close Time']}")
        data['Signal'] = "Buy"
        data["Close Price"] = df.iloc[-1]["Close"]
        data["Target"] = df.iloc[-1]["Close"] + target*df.iloc[-1]["Close"]/100
        data["Stop Loss"] = df.iloc[-1]["Close"] - sl*df.iloc[-1]["Close"]/100
        cron.send(data)
    if df.iloc[-1]["Sell"] == 1:
        print(f"Sell Signal at {df.iloc[-1]['Close Time']}")
        data['Signal'] = "Sell"
        data["Close Price"] = df.iloc[-1]["Close"]
        data["Target"] = df.iloc[-1]["Close"] - target*df.iloc[-1]["Close"]/100
        data["Stop Loss"] = df.iloc[-1]["Close"] + sl*df.iloc[-1]["Close"]/100
        cron.send(data)
    done = True

Startng sync...
Stopping
Start: 1672941599999.0
 Stop: 1672941660191.0
Last candle: 2023-01-05 17:59:59.999000
Current Time: 2023-01-05 23:31:00.191000
Dataframe End: 2023-01-05 17:44:59.999000
Open Time                              2023-01-05 17:45:00
Open                                               1246.98
High                                               1247.38
Low                                                1246.36
Close                                              1246.79
Volume                                             17316.5
Close Time                      2023-01-05 17:59:59.999000
Quote asset volumne                                    NaN
Total Trades                                           NaN
Buyer Base Volume                                      NaN
Taker buy quote asset volume                           NaN
Ignore                                                   0
Quote Asset Volume                              2.1591e+07
Number of Trades                       

In [11]:
# sync = BinanceSync(df,symbol,interval)
# sync.sync()
# sync.df.tail()

In [5]:
df = sync.df

In [5]:
1672940397137 - 1672940699999

-302862